In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time

In [2]:
year = ['2016','2017','2018','2019','2020']
'''
ㅇ영화 고유 번호 :
ㅇ리뷰 유무 : https://movie.naver.com/movie/bi/mi/review.nhn?code=140265&page=1 #151587
ㅇ리뷰 개수 : 파악 for문 돌릴 때 필요
ㅇ리뷰 링크 : https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4225025&code=140265&order=#tab

'''

'\nㅇ영화 고유 번호 :\nㅇ리뷰 유무 : https://movie.naver.com/movie/bi/mi/review.nhn?code=140265&page=1 #151587\nㅇ리뷰 개수 : 파악 for문 돌릴 때 필요\nㅇ리뷰 링크 : https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4225025&code=140265&order=#tab\n\n'

In [3]:
year = [i for i in range(2010,2022)]
year_num = [568,811,878,1219,1518,1410,1172,1051,999,850,731,510]

In [ ]:
for i in range(len(year)):
    a = int(year_num[i] / 20) +2


In [42]:
# 영화 고유 번호 크롤링 / 제목
title_list = []
url_list = []
for i in range(1, int(year_num[1] / 20) +2):
    url = f"https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open={2013}&page={i}"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    title = soup.select('ul.directory_list > li > a')

    for k in title:
        keyword = k.text
        title_list.append(keyword) 

    for href in soup.find("ul", class_="directory_list").find_all("li"):
        list = href.find("a")["href"]
        if 'code' in list :
            url_list.append(list)
    if i % 10 == 0 :
         print(i,'페이지 완료 /', end='')
#print(url_list)
print('')
print(year[3], '년도 영화 제목, href 파악')

10 페이지 완료 /20 페이지 완료 /30 페이지 완료 /40 페이지 완료 /50 페이지 완료 /60 페이지 완료 /
2013 년도 영화 제목, href 파악


In [ ]:
print(url_list)

In [43]:
# 영화 고유 번호 추출
code_list = []
for url in url_list:  #(len(url_list)):
    m_num = re.findall('\d+',url)
    code_list.append(m_num[0])
    
print(len(code_list))

1219


In [44]:
# 영화 리뷰 개수 크롤링
review_cnt_list = []
cnt = 1
for i in code_list:
    url = f"https://movie.naver.com/movie/bi/mi/review.nhn?code={i}"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    try : 
        r_num = soup.select_one('span.cnt > em').text
    except :
        r_num = 0
    if cnt % 10 == 0 :
        print('.', end='')
    review_cnt_list.append(r_num)
    
    cnt += 1
print('완료') 
#print(review_cnt_list)

.........................................................................................................................완료


In [45]:
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()

In [31]:
len(review_cnt_list)

878

In [ ]:
#숫자에 , 제거
for i in range(len(review_cnt_list)) :
    review_cnt_list[i] = re.sub(',','',str(review_cnt_list[i]))
print(review_cnt_list)

In [10]:
# 각 영화 리뷰 고유 번호 크롤링 2번째 영화 예시

review_num_list = []

for j in range(1, ((int(review_cnt_list[1])-1) // 10)+2) :
    print(j)
    url = f'https://movie.naver.com/movie/bi/mi/review.nhn?code=140265&page={j}'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    
    if j == 14 :
        time.sleep(1)
    for href in soup.find("ul", class_="rvw_list_area").find_all("li"):
        #list = href.find("a").split('showReviewDetail')
        list = href.find("a")
        list = re.findall('\(([^)]+)',str(list))[1]
        review_num_list.append(list)
    print(len(review_num_list))
    print('='*30)
    

print(review_num_list)


[]


In [55]:
len(review_num_list)

181

In [ ]:
#리뷰 텍스트 가져오기 - 예시
url = 'https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4225907&code=140265&order=#tab'
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
review = soup.select('div.user_tx_area')
review = re.sub('(<([^>]+)>)', '', str(review))
print(review)

In [ ]:
code_list.index('83697')

In [50]:
code_list[620]

'87163'

In [ ]:
#전체 코드 시작 2

"""
고유번호,제목   / code_list, title_list
리뷰의 개수 / review_cnt_list
고유번호-리뷰의 고유번호  /review_num_list (중간에 생성) url1
글 가져오기 / review_list / url2
"""

review_list = []
titles = [] # 칼럼 생성 위해서
cnt = 0
for i in range(1, len(code_list)) :
    num_id = code_list[i]
    title = title_list[i]
    
    #리뷰 고유 번호 파악
    review_num_list = []
    
    #리뷰 개수는 50개만 
    if int(review_cnt_list[i]) > 50:
        review_cnt_list[i] = '50'
    
    for j in range(1, ((int(review_cnt_list[i])-1) // 10)+2) :
        url = f'https://movie.naver.com/movie/bi/mi/review.nhn?code={num_id}&page={j}'
        req = requests.get(url)
        soup = BeautifulSoup(req.text, "html.parser")

        for href in soup.find("ul", class_="rvw_list_area").find_all("li"):
            #list = href.find("a").split('showReviewDetail')
            list = href.find("a")
            try :
                list = re.findall('\(([^)]+)',str(list))[1]
                review_num_list.append(list)
            except : 
                pass

            
    #리뷰 긁어오기

    for r in review_num_list :
        url = f'https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid={r}&code={num_id}&order=#tab'
        req = requests.get(url)
        soup = BeautifulSoup(req.text, "html.parser")
        review = soup.select('div.user_tx_area')
        review = re.sub('(<([^>]+)>)', '', str(review))
        review_list.append(review)
        titles.append(title)
    print(num_id," : ",title,' 완료/ ', end='')    
    cnt += 1
    if cnt % 100 == 0 :
        print('')
        print(cnt,'번째 영화 완료')
df= pd.DataFrame({'titles': titles, 'reviews': review_list})
df.info()
count=0
sentences = []
for sentence in df.reviews:
    count += 1
    if count % 100 == 0 :
        print('.', end='')
    if count % 1000 == 0 :
        print('/',end='') #줄바꿈
    sentence = re.sub('[^가-힣 ]','',sentence)
    sentences.append(sentence)
df= pd.DataFrame({'titles': titles, 'reviews': sentences})
df.to_csv('reviews_{2013}.csv')    
print('완료')

In [20]:
df= pd.DataFrame({'titles': titles, 'reviews': review_list})
df.info()
count=0
sentences = []
for sentence in df.reviews:
    count += 1
    if count % 100 == 0 :
        print('.', end='')
    if count % 1000 == 0 :
        print('/',end='') #줄바꿈
    sentence = re.sub('[^가-힣 ]','',sentence)
    sentences.append(sentence)

len(sentences)

df.to_csv('reviews_2010.csv')
# print('완료')
# #dfencode('UTF-8','ignore').decode('UTF-8')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16383 entries, 0 to 16382
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titles   16383 non-null  object
 1   reviews  16383 non-null  object
dtypes: object(2)
memory usage: 256.1+ KB
........../........../........../........../........../........../........../........../........../........../........../........../........../........../........../........../...

In [198]:
df= pd.DataFrame({'titles': titles3, 'reviews': sentences})
df.to_csv('reviews_2016_.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21762 entries, 0 to 21761
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titles   21762 non-null  object
 1   reviews  21762 non-null  object
dtypes: object(2)
memory usage: 340.2+ KB


In [141]:
# 왜 오류가 났을까?

review_num_list = []
for j in range(1, (50// 10)+2) :
    url = f'https://movie.naver.com/movie/bi/mi/review.nhn?code={114237}&page={j}'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    for href in soup.find("ul", class_="rvw_list_area").find_all("li"):
        #list = href.find("a").split('showReviewDetail')
        list = href.find("a")
        print(j, list)
        try :
            list = re.findall('\(([^)]+)',str(list))[1]
            review_num_list.append(list)
        except : 
            pass
        

1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4201037)"><strong>&lt;드레스메이커 , 2016&gt; 시작은 창대하나 그 끝은 미약하리라</strong></a>
1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4255753)"><strong>드레스메이커를 보고</strong></a>
1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4194331)"><strong>[시사회] 영화 드레스메이커 후기</strong></a>
1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4186965)"><strong>(2016.01.26) [영화|후기]「드레스메이커」- 살인자로 낙인찍었던 마을로 돌아온 그녀, 가혹한 그녀의 운명 그리고 선택! (+ 관객과의 대화 GV 모습 등)</strong></a>
1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4205376)"><strong>스포有 ) 복수극에 로맨스가 웬말인가</strong></a>
1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4223160)"><strong>드레스메이커 / 20160224</strong></a>
1 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4182361)"><strong>드레스메이커(2015)

4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4201988)"><strong>영화드레스메이커보고왔어요!</strong></a>
4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4185981)"><strong>※약스포포함※ 흔하고 뻔한 복수극은 가라!!! &lt;드레스메이커&gt;</strong></a>
4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4585804)"><strong>이슈메이커, 트러블메이커, 드레스메이커 틸리.</strong></a>
4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4219738)"><strong>[드레스메이커] 환상동화 같은 이야기</strong></a>
4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4203465)"><strong>영화 드레스 메이커 (스포 有) - 열정적이고 시원하면서 낭만적인 마지막 장면</strong></a>
4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4210545)"><strong>[드레스메이커] 순박하지 않은 시골 사람들의 무서운 이야기</strong></a>
4 <a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4202173)"><strong>드레스메이커-화려함과 잔혹함의 조화</strong></a>
4 

In [68]:
#리뷰 고유번호 구하기 / 오류 뜨는 이유??
url = "https://movie.naver.com/movie/bi/mi/review.nhn?code=140265&page=13"
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
print("The href links are :")

list2 = []
for href in soup.find("ul", class_="rvw_list_area").find_all("li"):
    #list = href.find("a").split('showReviewDetail')
    list = href.find("a")
    print(list)
    list = re.findall('\(([^)]+)',str(list))[1]
    list2.append(list)
print(list2)

The href links are :
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4212132)"><strong>&lt;13 시간&gt;</strong></a>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4223711)"><strong>영화"13시간" 보는 내내 숨쉴 수가 없습니다.</strong></a>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4229769)"><strong>영화 13시간 X 아, 짜증나 </strong></a>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4219558)"><strong>마이클 베이 &lt;더록&gt; 시절 초심으로 돌아오는건가?</strong></a>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4187225)"><strong>영화 &lt;13시간&gt; 예고편</strong></a>
['4212132', '4223711', '4229769', '4219558', '4187225']


In [53]:
review_list = []
titles = [] # 칼럼 생성 위해서
cnt = 0
#87163 /620

for j in range(1, 50) : 
    url = f'https://movie.naver.com/movie/bi/mi/review.nhn?code={87163}&page={j}'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    
    print(j, '페이지')
    for href in soup.find("ul", class_="rvw_list_area").find_all("li"):
        #list = href.find("a").split('showReviewDetail')
        try :
            list = href.find("a")
            list = re.findall('\(([^)]+)',str(list))[1]
            print(list)
            review_num_list.append(list)
        except :
            continue


1 페이지
2990153
2992055
2774467
2993777
2551108
4353345
2626153
2629025
3578558
3059776
2 페이지
3809512
2626994
3550350
3829650
2562529
2626322
2515636
2597501
3591449
3450113
3 페이지
2997032
2991777
2993502
2989424
4260827
2993780
2990149
2990957
3028143
2630178
4 페이지
2969773
2988495
3440376
5 페이지
2992436
4341644
3012113
4176305
4566260
4581685
4341178
4158238
4391727
2711530
6 페이지
2994510
2992403
2811596
2989877
3117822
3056301
2813634
2993341
2999793
3146993
7 페이지
3000826
3264345
3026091
3097548
3017327
3044123
4174607
4426657
4131752
4651041
8 페이지
2659383
2991976
2710221
2979690
4587518
2994971
2992063
2989384
3025185
2989796
9 페이지
3619812
3114630
2992060
3007306
3170889
3002048
3238035
2986628
3021337
2997039
10 페이지
2998722
3205257
3002918
3004476
3051467
3271629
3491311
3077623
3800710
3548921
11 페이지
4236900
4160994
4145715
4277174
4656751
3007903
2989737
2994466
3009730
2992959
12 페이지
2991612
3006487
3247834
3013027
3201801
2996565
2994783
3239208
3192501
3098793
13 페이지
2734996
303998

KeyboardInterrupt: 